# WorkShop 2

Let's play with pandas and answer "stakeholders" _questions_.

Problems $2-13$ rely on your `Problem 1` answer.

## Problem  1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of __energy_df__.

Keep in mind that this is an _Excel file_, and not a _comma separated values file_. Also, make sure to _exclude_ the footer and header information from the datafile. The first _two columns_ are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

```python
['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
```

Convert `Energy Supply` to _gigajoules_ (there are $1000000$ _gigajoules_ in a _petajoule_**). For all countries which have missing data (_e.g._ data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (_for use in later questions_):

```bash
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

There are also several countries with parenthesis in their name. Be sure to _remove these_, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with _Numeric digits_ in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a _csv_ containing countries' GDP from $1960$ to $2015$ from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame __gdp_df__. 

Make sure to _skip the header_, and _rename_ the following list of countries:

```bash
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finally, load the [Scimago Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their _journal contributions_ in the aforementioned area. Call this DataFrame __scimago_df__.

Join the _thrgoee datasets_: _energy\_df_, _gdp\_df_ and _scimago\_df_, into a new dataset (using the _intersection_ of _country names_). Use only the _last_ $10$ years ($2006-2015$) of __GDP data__ and only the _top 15 countries_ by __Scimagojr 'Rank'__ (_Rank 1 through 15_). 

The _index_ of this DataFrame should be the __name__ of the country, and the _columns_ should be:
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']`.

_This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank"._

In [61]:
import pandas as pd
import numpy as np

def generate_energy_df():
    energy_df = pd.read_excel('assets/Energy Indicators.xls', skipfooter=38, skiprows=17, usecols=[2, 3, 4, 5])
    energy_df.columns = ['Country Names', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy_df = energy_df.replace('...', np.nan)
    energy_df['Energy Supply'] = energy_df['Energy Supply'] * 1e6
    energy_df['Country Names'] = energy_df['Country Names'].str.replace(r'\s*\(.*?\)', '', regex=True)
    energy_df['Country Names'] = energy_df['Country Names'].str.replace(r'\d', '', regex=True)
    country_replacements = {
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"
    }
    energy_df['Country Names'] = energy_df['Country Names'].replace(country_replacements)
    energy_df = energy_df.set_index('Country Names')
    return energy_df

In [62]:
def get_GDP_df():
    gdp_df = pd.read_csv('assets/world_bank.csv', skiprows=4)
    country_replacements = {
        "Korea, Rep.": "South Korea", 
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong"
    }
    gdp_df['Country Name'] = gdp_df['Country Name'].replace(country_replacements)
    columns_needed = ['Country Name', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    gdp_df = gdp_df[columns_needed]
    gdp_df = gdp_df.set_index('Country Name')
    return gdp_df

In [63]:
def get_scimago_df():
    scimago_df = pd.read_excel('assets/scimagojr-3.xlsx')
    scimago_df = scimago_df.head(15)
    scimago_df = scimago_df.set_index('Country')
    return scimago_df

In [64]:
def answer_one():
    energy_df = generate_energy_df()
    gdp_df = get_GDP_df()
    scimago_df = get_scimago_df()

    combined_df = pd.merge(scimago_df, energy_df, how='inner', left_index=True, right_index=True)
    combined_df = pd.merge(combined_df, gdp_df, how='inner', left_index=True, right_index=True)
    combined_df = combined_df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index',
                               'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010',
                               '2011', '2012', '2013', '2014', '2015']]
    combined_df = combined_df.sort_values('Rank')
    return combined_df

In [65]:
# Test the function
final_df = answer_one()

# Display the final DataFrame
print(final_df)

# Example of test cases
assert type(final_df) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert final_df.shape == (15, 20), "Q1: Your DataFrame should have 20 columns and 15 entries!"

                    Rank  Documents  Citable documents  Citations  \
China                  1     127050             126767     597237   
United States          2      96661              94747     792274   
Japan                  3      30504              30287     223024   
United Kingdom         4      20944              20357     206091   
Russian Federation     5      18534              18301      34266   
Canada                 6      17899              17620     215003   
Germany                7      17027              16831     140566   
India                  8      15005              14841     128763   
France                 9      13153              12973     130632   
South Korea           10      11983              11923     114675   
Italy                 11      10964              10794     111850   
Spain                 12       9428               9330     123336   
Iran                  13       8896               8819      57470   
Australia             14       883

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [66]:
# space for proffesor tests

## Problem 2

The _Problem 1_ joined three datasets then reduced this to just the top $15$ entries. When you joined the datasets, but before you reduced this to the top $15$ items, how many entries did you lose?

_This function should return a single number._

In [67]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [68]:
def answer_two():
    energy_df = generate_energy_df()
    gdp_df = get_GDP_df()
    scimago_df = get_scimago_df()

    # Número de entradas en cada DataFrame original
    num_entries_energy = energy_df.shape[0]
    num_entries_gdp = gdp_df.shape[0]
    num_entries_scimago = scimago_df.shape[0]

    # Unión de los DataFrames sin filtrar los primeros 15 elementos
    combined_df = pd.merge(scimago_df, energy_df, how='inner', left_index=True, right_index=True)
    combined_df = pd.merge(combined_df, gdp_df, how='inner', left_index=True, right_index=True)

    # Número de entradas después de la combinación
    num_entries_combined = combined_df.shape[0]

    # Calcular el número de entradas perdidas
    total_initial_entries = len(set(energy_df.index) | set(gdp_df.index) | set(scimago_df.index))
    entries_lost = total_initial_entries - num_entries_combined

    return entries_lost

In [69]:

# Test the function
print(answer_two())

# Example of test cases
assert type(answer_two()) == int, "Q2: You should return an int number!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


290


In [70]:
# space for proffesor tests

## Problem 3

What are the top $15$ countries for average GDP over the last $10$ years?

_This function should return a Series named `avg_gdp` with $15$ countries and their average GDP sorted in descending order._

In [71]:
def answer_three():
    # Retrieve the DataFrame from the answer_one function
    df = answer_one()

    # Calculate the average GDP for the years 2006 to 2015
    avg_gdp = df.loc[:, '2006':'2015'].mean(axis=1)

    # Sort the average GDP values in descending order
    avg_gdp_sorted = avg_gdp.sort_values(ascending=False)

    return avg_gdp_sorted

# Get and display the top 15 values of the sorted average GDP
print(answer_three().head(15))


United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64


C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [72]:
# example of test cases
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [73]:
# space for proffesor tests

# Problem 4

By how much had the GDP changed over the $10$ year span for the country with the $6-th$ largest average GDP?

_This function should return a single number._

In [74]:
def answer_four():
    # Get the DataFrames from the functions answer_three and answer_one
    answer_three_df = answer_three()
    answer_one_df = answer_one()
    
    # Create a new DataFrame with the 'GDP' column from the DataFrame obtained from answer_three
    GDP_df = pd.DataFrame(answer_three_df, columns=['GDP'])
    
    # Get the name of the country in the sixth position
    sixth_country = GDP_df.index[5]
    
    # Calculate the variation between the years 2015 and 2006 for the sixth country
    variation = answer_one_df.loc[sixth_country, '2015'] - answer_one_df.loc[sixth_country, '2006']
    
    return float(variation)

# Call the function to verify that it works correctly
print(answer_four())

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


246702696075.3999


In [75]:
# example of test cases
assert type(answer_four()) == float, "Q4: You should return a single number!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [76]:
# space for proffesor tests


## Problem 5

What is the _mean energy supply per capita_?

_This function should return a single number._

In [77]:
def answer_five():
    # Get the DataFrame from the function answer_one
    answer_one_df = answer_one()
    
    # Calculate the mean of the 'Energy Supply per Capita' column
    mean_energy = answer_one_df['Energy Supply per Capita'].mean()
    
    return float(mean_energy)

# Call the function to verify that it works correctly
print(answer_five())

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


157.6


In [78]:
# example of test cases
assert type(answer_five()) == float, "Q5: You should return a single number!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [79]:
# space for proffesor tests

## Problem 6

What __country__ has the _maximum % Renewable_ and what is the _percentage_?

_This function should return a tuple with the name of the country and the percentage._

In [80]:
def answer_six():
    # Get the DataFrame from the function answer_one
    answer_one_df = answer_one()
    
    # Find the country with the maximum '% Renewable' value
    max_renewable_country = answer_one_df['% Renewable'].idxmax()
    
    # Get the maximum '% Renewable' value for that country
    max_renewable_percent = answer_one_df.loc[max_renewable_country, '% Renewable']
    
    # Return the country and its maximum '% Renewable' value as a tuple
    return (max_renewable_country, max_renewable_percent)

# Call the function to verify that it works correctly
print(answer_six())

('Brazil', np.float64(69.64803))


C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [81]:
# example of test cases
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [82]:
# space for proffesor tests

## Problem 7

Create a new column that is the _ratio of Self-Citations to Total Citations_. 
What is the _maximum_ value for this new column, and what _country_ has the _highest ratio_?

_This function should return a tuple with the name of the country and the ratio._

In [83]:
def answer_seven():
    # Get the DataFrame from the function answer_one
    answer_one_df = answer_one()
    
    # Calculate the ratio of 'Self-citations' to 'Citations' and create a new column 'Ratio'
    answer_one_df['Ratio'] = answer_one_df['Self-citations'] / answer_one_df['Citations']
    
    # Find the country with the maximum 'Ratio'
    radio_index = answer_one_df['Ratio'].idxmax()
    
    # Get the maximum 'Ratio' value
    radio_num = answer_one_df['Ratio'].max()
    
    # Return the country and its maximum 'Ratio' value as a tuple
    return (radio_index, radio_num)

# Call the function to verify that it works correctly
print(answer_seven())

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


('China', np.float64(0.6893126179389422))


In [84]:
# example of test cases
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [85]:
# space for proffesor tests

## Problem 8

Create a column that estimates the _population using Energy Supply_ and _Energy Supply per capita_. 
What is the _third_ most populous country according to this estimate?

_This function should return the name of the country._

In [86]:
def answer_eight():
    # Get the DataFrame using the answer_one() function
    answer_one_df = answer_one()
    
    # Calculate the population and add it to the DataFrame
    answer_one_df['Population'] = answer_one_df['Energy Supply'] / answer_one_df['Energy Supply per Capita']
    
    # Sort the DataFrame by population in descending order
    sorted_df = answer_one_df.sort_values(by='Population', ascending=False)
    
    # Get the name of the country with the third highest population
    third_largest_population_country = sorted_df.iloc[2].name
    
    return third_largest_population_country

# Call the function to verify its functionality
print(answer_eight())


United States


C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [87]:
# example of test cases
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [88]:
# space for proffesor tests

## Problem 9

Create a column that estimates the _number of citable documents per person_. 

What is the correlation between the _number of citable documents per capita_ and the _energy supply per capita_? Use the `.corr()` method, (Pearson's correlation).

_This function should return a single number._

In [89]:
def answer_nine():
    # Get the DataFrame using the answer_one() function
    answer_one_df = answer_one()
    
    # Calculate the population by country
    answer_one_df['Population'] = answer_one_df['Energy Supply'] / answer_one_df['Energy Supply per Capita']
    
    # Calculate the Citable Documents per Capita
    answer_one_df['CitableDocuments per Capita'] = answer_one_df['Citable documents'] / answer_one_df['Population']
    
    # Compute the correlation between Citable Documents per Capita and Energy Supply per Capita
    correlation = answer_one_df['CitableDocuments per Capita'].corr(answer_one_df['Energy Supply per Capita'])
    
    return correlation

# Call the function to verify its functionality
print(answer_nine())


0.7940010435442946


C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [90]:
# example of test cases
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [91]:
# space for proffesor tests

## Problem 10

Create a new column with a $1$ if the _country's % Renewable_ value is at or above the median for all countries in the _top 15_, and a $0$ if the _country's % Renewable_ value is below the median.

_This function should return a series named `high_renew` whose index is the country name sorted in ascending order of rank._

In [92]:
def answer_ten():
    # Get the DataFrame using the answer_one() function
    answer_one_df = answer_one()
    
    # Sort the DataFrame by 'Rank' in ascending order and select the top 15 countries
    top_15_df = answer_one_df.head(15)
    
    # Calculate the median value of the '% Renewable' column for the top 15 countries
    median_renewable_top_15 = top_15_df['% Renewable'].median()
    
    # Create a new column 'High Renewable' with 'Yes' if '% Renewable' is at or above the median, otherwise 'No'
    answer_one_df['High Renewable'] = answer_one_df.apply(lambda x: 'Yes' if x['% Renewable'] >= median_renewable_top_15 else 'No', axis=1)
    
    # Sort the DataFrame by 'Rank' in ascending order
    sorted_df = answer_one_df.sort_values(by='Rank', ascending=True)
    
    # Create the 'high_renew' Series with country names as the index
    high_renew = sorted_df['High Renewable']
    
    return high_renew

# Check the result
answer_ten()


C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


China                 Yes
United States          No
Japan                  No
United Kingdom         No
Russian Federation    Yes
Canada                Yes
Germany               Yes
India                  No
France                Yes
South Korea            No
Italy                 Yes
Spain                 Yes
Iran                   No
Australia              No
Brazil                Yes
Name: High Renewable, dtype: object

In [93]:
# example of test cases
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [94]:
# space for proffesor tests

## Problem 11

Use the following dictionary to group the _Countries by Continent_, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the _sum_, _mean_, and _std deviation_ for the estimated population of each country.

```python
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}
```

_This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`_

In [95]:
def answer_eleven():
    # Get the DataFrame using the answer_one() function
    answer_one_df = answer_one()
    
    # Define the ContinentDict to map countries to continents
    ContinentDict = {
        'China': 'Asia', 
        'United States': 'North America', 
        'Japan': 'Asia', 
        'United Kingdom': 'Europe', 
        'Russian Federation': 'Europe', 
        'Canada': 'North America', 
        'Germany': 'Europe', 
        'India': 'Asia',
        'France': 'Europe', 
        'South Korea': 'Asia', 
        'Italy': 'Europe', 
        'Spain': 'Europe', 
        'Iran': 'Asia',
        'Australia': 'Australia', 
        'Brazil': 'South America'
    }
    
    # Add Continent column to the DataFrame
    answer_one_df['Continent'] = answer_one_df.index.to_series().map(ContinentDict)
    
    # Calculate Population if not already present
    answer_one_df['Population'] = answer_one_df['Energy Supply'] / answer_one_df['Energy Supply per Capita']
    
    # Group by Continent and calculate the required statistics
    continent_agrupation = answer_one_df.groupby('Continent')['Population'].agg(['size', 'sum', 'mean', 'std'])
    
    # Reorder the DataFrame index as specified
    ordered_index = ['Asia', 'Australia', 'Europe', 'North America', 'South America']
    continent_agrupation = continent_agrupation.reindex(ordered_index)
    
    return continent_agrupation

# Call the function to see the result
answer_eleven()


C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


In [96]:
# example of test cases
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.

In [97]:
# space for proffesor tests

## Problem 12

Cut (check `.cut()` from Pandas) _% Renewable_ into $5$ bins, but in the same column. Group _Top15_ by the _Continent_, as well as these new _% Renewable_ bins. How many countries are in each of these groups? You could check `.size()` as aggregation in group by.

_This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable` (in string datatype). Do not include groups with no countries._

In [98]:
def answer_twelve():
    df = answer_one()
    ContinentDict = {
        'China': 'Asia', 
        'United States': 'North America', 
        'Japan': 'Asia', 
        'United Kingdom': 'Europe', 
        'Russian Federation': 'Europe', 
        'Canada': 'North America', 
        'Germany': 'Europe', 
        'India': 'Asia',
        'France': 'Europe', 
        'South Korea': 'Asia', 
        'Italy': 'Europe', 
        'Spain': 'Europe', 
        'Iran': 'Asia',
        'Australia': 'Australia', 
        'Brazil': 'South America'
    }
    
    df['Continent'] = df.index.map(ContinentDict)
    df['Renewable_Bin'] = pd.cut(df['% Renewable'], bins=5)
    top_15_df = df.sort_values(by='Rank').head(15)
    
    agrupation = top_15_df.groupby(['Continent', 'Renewable_Bin']).size()
    
    # Debugging output
    print("Top 15 DataFrame:")
    print(top_15_df)
    print("Renewable Bins:")
    print(df['Renewable_Bin'].value_counts())
    print("Grouped Data:")
    print(agrupation)
    
    return agrupation

# Call the function to debug
answer_twelve()


C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1524911742.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agrupation = top_15_df.groupby(['Continent', 'Renewable_Bin']).size()


Top 15 DataFrame:
                    Rank  Documents  Citable documents  Citations  \
China                  1     127050             126767     597237   
United States          2      96661              94747     792274   
Japan                  3      30504              30287     223024   
United Kingdom         4      20944              20357     206091   
Russian Federation     5      18534              18301      34266   
Canada                 6      17899              17620     215003   
Germany                7      17027              16831     140566   
India                  8      15005              14841     128763   
France                 9      13153              12973     130632   
South Korea           10      11983              11923     114675   
Italy                 11      10964              10794     111850   
Spain                 12       9428               9330     123336   
Iran                  13       8896               8819      57470   
Australia       

Continent      Renewable_Bin   
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Australia      (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
               (42.701, 56.174]    0
               (56.174, 69.648]    0
North America  (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
South America  (2.212, 15.753]     0
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
dtype: int64

In [99]:
# example of test cases
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1524911742.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agrupation = top_15_df.groupby(['Continent', 'Renewable_Bin']).size()


Top 15 DataFrame:
                    Rank  Documents  Citable documents  Citations  \
China                  1     127050             126767     597237   
United States          2      96661              94747     792274   
Japan                  3      30504              30287     223024   
United Kingdom         4      20944              20357     206091   
Russian Federation     5      18534              18301      34266   
Canada                 6      17899              17620     215003   
Germany                7      17027              16831     140566   
India                  8      15005              14841     128763   
France                 9      13153              12973     130632   
South Korea           10      11983              11923     114675   
Italy                 11      10964              10794     111850   
Spain                 12       9428               9330     123336   
Iran                  13       8896               8819      57470   
Australia       

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1524911742.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agrupation = top_15_df.groupby(['Continent', 'Renewable_Bin']).size()


AssertionError: Q12: Wrong result numbers!

In [ ]:
# space for proffesor tests

## Problem 13

Convert the _Population Estimate_ series to a string with thousands separator (using _dots_). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12.345.678,90

_This function should return a series `pop_est` whose index is the country name and whose values are the population estimate string_

In [100]:
def answer_thirteen():
    # Get the DataFrame using the answer_one() function
    df = answer_one()
    
    # Calculate the Population
    df['Population'] = df['Energy Supply'] / df['Energy Supply per Capita']
    
    # Format Population to a string with 10 decimal places
    pop_est = df['Population'].apply(lambda x: f"{x:,.10f}")
    
    return pop_est

# Call the function to verify it produces the expected result
answer_thirteen()


C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


China                 1,367,645,161.2903225422
United States           317,615,384.6153846383
Japan                   127,409,395.9731543660
United Kingdom           63,870,967.7419354841
Russian Federation      143,500,000.0000000000
Canada                   35,239,864.8648648635
Germany                  80,369,696.9696969688
India                 1,276,730,769.2307691574
France                   63,837,349.3975903615
South Korea              49,805,429.8642533943
Italy                    59,908,256.8807339445
Spain                    46,443,396.2264150977
Iran                     77,075,630.2521008402
Australia                23,316,017.3160173148
Brazil                  205,915,254.2372881472
Name: Population, dtype: object

In [101]:
# example of test cases
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"

C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)
C:\Users\manue\AppData\Local\Temp\ipykernel_2432\1853173759.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [ ]:
# space for proffesor tests